# Creating a RAG model with Langchain and Openrouter

Kelompok 1 Algoritma Deep Learning
Anggota Kelompok :
1. A IAS Falah Surya Gemilang
2. Aldo Rizky Ramadhan
3. Faiz Rizki Azmi
4. Muhamad Ibnu Khaidar Hafiz
5. Siti Asma Tomu

In [1]:
## install langchain
%pip install -qU langchain-nvidia-ai-endpoints

Note: you may need to restart the kernel to use updated packages.


In [2]:
import getpass
import os

os.environ["NVIDIA_API_KEY"] = getpass.getpass()

from langchain_nvidia_ai_endpoints import ChatNVIDIA

llm = ChatNVIDIA(model="meta/llama3-70b-instruct")

In [4]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_nvidia import NVIDIAEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Load, chunk and index the contents of the blog.
loader = WebBaseLoader(
    web_paths=("https://id.wikipedia.org/wiki/Universitas_Gunadarma",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=NVIDIAEmbeddings())

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("When was gunadarma founded?")

IndexError: list index out of range in upsert.

In [ ]:
# Third Cell - Text Splitting
def split_documents(documents: List) -> List:
    """
    Split documents into chunks
    """
    # For markdown documents
    headers_to_split_on = [
        ("#", "Header 1"),
        ("##", "Header 2"),
        ("###", "Header 3"),
    ]
    markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
    
    # General text splitter
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len,
        separators=["\n\n", "\n", " ", ""]
    )
    
    splits = []
    for doc in documents:
        # Try markdown splitting first
        try:
            header_splits = markdown_splitter.split_text(doc.page_content)
            text_splits = text_splitter.split_documents(header_splits)
        except:
            # Fall back to regular splitting if not markdown
            text_splits = text_splitter.split_documents([doc])
        splits.extend(text_splits)
    
    return splits


In [ ]:
# Fourth Cell - Vector Store Creation
def create_vector_store(documents: List):
    """
    Create and return a vector store from the documents
    """
    embeddings = HuggingFaceEmbeddings(
        model_name="all-MiniLM-L6-v2",
        model_kwargs={'device': 'cuda'},
        encode_kwargs={'device': 'cuda', 'batch_size': 32}
    )
    
    vector_store = Chroma.from_documents(
        documents=documents,
        embedding=embeddings,
        persist_directory="./chroma_db"
    )
    
    return vector_store


In [ ]:
# Fifth Cell - RAG Chain Creation
def create_rag_chain(vector_store):
    """
    Create and return a RAG chain
    """
    retriever = vector_store.as_retriever(
        search_type="mmr",
        search_kwargs={
            "k": 5,
            "fetch_k": 10
        }
    )
    
    def format_docs(docs):
        return "\n\n".join(doc.page_content for doc in docs)
    
    # Get the RAG prompt from LangChain hub
    prompt = hub.pull("rlm/rag-prompt")
    
    # Create the RAG chain
    rag_chain = (
        {"context": retriever | format_docs, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )
    
    return rag_chain
